In [10]:
class Bridge :
    def __init__(self, name , id_ , flag ):
        self.name = name
        self.id = id_
        
        self.connections = set()
        self.root = self
        self.dist = 0
        
        self.rp = None
        self.dp = set() 
        self.np = set()
        self.next = None
        
        self.change = False
        self.forward = False
        self.final_config = []
        
        self.flag = flag
        self.trace = []
        
        
    def send(self,t):
        if self.root is self or self.forward :
            if self.flag:
                self.trace.append(f'{t} s {self} ({self.root} {self.dist} {self})')
            
            for lan in self.connections:
                
                lan.forward((self.root,self.dist,self),t)
        
        self.change = False
        self.forward = False
    
    def connect(self,lan):
        self.connections.add(lan)
        self.dp.add(lan) 
    
    def receive(self,msg,lan,t):
        root,dist,bridge = msg
        
        if self.flag:
            self.trace.append(f'{t+1} r {self} ({root} {dist} {bridge})')
            
        if self.dist > dist or (self.dist == dist and self.id > bridge.id):
            if lan in self.dp:
                self.dp.remove(lan)
                self.change = True
        
        dist +=1
        
        if root.id > self.root.id or (root.id == self.root.id and dist > self.dist):
            return
        if self.next:
            if root.id == self.root.id and dist == self.dist and bridge.id > self.next.id:
                return
            
        self.change = True
        self.forward = True
        self.root = root
        self.dist = dist
        self.rp = lan
        self.next = bridge

    def __repr__(self):
        return self.name
    
    def finalize(self):
        config = []
        for con in self.connections:
            if con is self.rp:
                config.append(f'{con}-RP')
            
            elif con in self.dp :
                config.append(f'{con}-DP')
            
            else:
                self.np.add(con)
                config.append(f'{con}-NP')
                
        self.final_config = config
        return config
    
class LAN:
    
    def __init__(self,name):
        self.name = name
        self.connections = set()
        
    
    def connect(self,bridge):
        self.connections.add(bridge)
        
    def forward(self,message,t):
        sender = message[2]
        
        for bridge in self.connections:
            
            if bridge is not sender:
                bridge.receive(message,self,t)
           
    def __repr__(self):
        return self.name
        

In [11]:
def learn(bridges,lans,flag):
    cond = True
    t=1
    
    while cond:
        
        for i in bridges:
            bridges[i].send(t)
        t+=1
        cond=False
        
        for j in bridges:
            if bridges[j].change:
                cond = True
                break
    if flag:
        trace = []
        
        for k in bridges:
            trace.extend(bridges[k].trace)
            bridges[k].trace.clear()
        print('\n'.join(sorted(trace)))
        
    for l in range(len(bridges)):
        print(f'{bridges[l].name}: ' + ' '.join(sorted(bridges[l].finalize())))

bridges = {}
lans = {}   

f = input() 
B = int(input())

    
for i in range(B):
    x = input()
    
    bridge = x.split()[0][:-1]
    ports =  x.split()[1:]
    
    bridges[i] = Bridge(bridge,i,f)
        
    for pt in ports :
        #print(p)
        if pt not in lans:
            lans[pt] = LAN(pt)
                
        bridges[i].connect(lans[pt])
        lans[pt].connect(bridges[i])
            
   
        
learn(bridges, lans , f)
 
            
            
            
            
            
            
            
            
            
            
        

1
5
B1: A G B
B2: G F
B3: B C
B4: C F E
B5: C D E
1 s B1 (B1 0 B1)
1 s B2 (B1 1 B2)
1 s B3 (B1 1 B3)
1 s B4 (B1 2 B4)
1 s B5 (B1 2 B5)
2 r B1 (B1 1 B2)
2 r B1 (B1 1 B3)
2 r B2 (B1 0 B1)
2 r B2 (B1 2 B4)
2 r B3 (B1 0 B1)
2 r B3 (B1 2 B4)
2 r B3 (B1 2 B5)
2 r B4 (B1 1 B2)
2 r B4 (B1 1 B3)
2 r B4 (B1 2 B5)
2 r B4 (B1 2 B5)
2 r B5 (B1 1 B3)
2 r B5 (B1 2 B4)
2 r B5 (B1 2 B4)
B1: A-DP B-DP G-DP
B2: F-DP G-RP
B3: B-RP C-DP
B4: C-NP E-DP F-RP
B5: C-RP D-DP E-NP
